In [1]:
import numpy as np
from fastai import *
import csv
from glob import glob
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import fastai
# from fastai.data.core import DataLoaders
# from fastai.vision.learner import *
from fastai.basics import *
from fastai.vision.core import *
from fastai.vision.data import *
from fastai.vision.augment import *
from fastai.vision import models
# from fastai.vision.learner import Learner
from fastai.vision import models
from torch import nn
import torch.nn.functional as F

In [2]:
torch.cuda.is_available()

True

In [3]:
# imgs = '/media/sarvagya/66e1ccd6-9d4b-4a61-8750-36e2e23d8988/solar/11/'
# days = (sorted(os.listdir(imgs)))
# files = []
# for day in days:
# #     print("#####################")
# #     print(sorted(glob(imgs+month+'/*.npz')))
#     img = sorted(glob(imgs+day+'/*.npz'))
#     for i in img:
#         files.append(i)

In [4]:
# len(files)

In [5]:
# column_name = ['image']
# with open('output.csv', 'w') as w:
#     write = csv.writer(w)
#     write.writerow(column_name)

In [6]:
# for file in files:
#     with open('output.csv', 'a') as w:
#         write = csv.writer(w)
#         write.writerow([file])

In [3]:
df = pd.read_csv('output.csv', error_bad_lines=False)

In [4]:
# df

In [5]:
X = list(df['image'])

In [6]:
tmp = np.load(X[0], allow_pickle=True)
img = tmp['x']
img = np.reshape(img,(1,img.shape[0], img.shape[1]))
img = torch.from_numpy(img).float()
print(img.shape)

torch.Size([1, 512, 512])


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, X, test_size=0.33, random_state=42)

In [8]:
type(X_train)

list

In [9]:
class data_gen(torch.utils.data.Dataset):
    def __init__(self, files):
        
        self.files = files
        
        
    def __getitem__(self, i):
        
        file1 = self.files[i]
        
        tmp = np.load(file1, allow_pickle=True)
        
        img = tmp['x']
        img = np.reshape(img,(1,img.shape[0], img.shape[1]))
        img = torch.from_numpy(img).float()
        
        return img

    def __len__(self): 
        
        return len(self.files)

In [10]:
train_ds = data_gen(X_train)
test_ds = data_gen(X_test)

In [11]:
batch_size = 16
dls = DataLoaders.from_dsets(train_ds, test_ds, bs=batch_size, device='cuda:0')

In [12]:
b = dls.one_batch()

In [13]:
b.size()

torch.Size([16, 1, 512, 512])

In [14]:
# learn = unet_learner(dls, models.resnet34)

In [15]:
# class Autoencoder(nn.Module):
#     def __init__(self):
#         super(Autoencoder, self).__init__()
#         # Input size: [batch, 3, 32, 32]
#         # Output size: [batch, 3, 32, 32]
#         self.encoder = nn.Sequential(
#             nn.Conv2d(1,12,4,stride=2,padding=1),
#             nn.ReLU(),
#             nn.Conv2d(12,24,4,stride=2,padding=1),
#             nn.ReLU(),
#             nn.Conv2d(24,48,4,stride=2,padding=1),
#             nn.ReLU(),
#             )
#         self.decoder = nn.Sequential(
#             nn.ConvTranspose2d(48,24,4,stride=2,padding=1),
#             nn.ReLU(),
#             nn.ConvTranspose2d(24,12,4,stride=2,padding=1),
#             nn.ReLU(),
#             nn.ConvTranspose2d(12,1,4,stride=2,padding=1),
#             nn.Sigmoid(),
#             )    
#         def encode(self, x): return self.encoder(x)    
        
#         def decode(self, x): return self.decoder(x)    
#         def forward(self, x):
#             encoded = self.encoder(x)
#             decoded = self.decoder(encoded)
#             return decoded
m = fastai.vision.models.resnet34(pretrained = False).cuda()

In [16]:
# autoencoder = Autoencoder()
m = nn.Sequential(*list(m.children())[:-3])

In [17]:
code_sz = 32

conv = nn.Conv2d(256, code_sz, kernel_size=(2,2)).cuda()

m.add_module('CodeIn',conv)

In [18]:
m._modules['0'] = nn.Conv2d(1, 64, kernel_size=(7,7),stride=2,padding=1).cuda()
# m._modules['1'] = nn.BatchNorm2d(batch_size, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True).cuda()
m

Sequential(
  (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(1, 1))
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, 

In [19]:
# autoencoder(b)
m(b).size()

torch.Size([16, 32, 31, 31])

In [20]:
# learn = Learner(data, autoencoder, loss_func = F.mse_loss)
class UpSample(nn.Module):
    def __init__(self,feat_in,feat_out,out_shape=None,scale=2):
        super().__init__()
        self.conv = nn.Conv2d(feat_in,feat_out,kernel_size=(3,3),stride=1,padding=1)
        self.out_shape,self.scale = out_shape,scale
        
    
    def forward(self,x):
        return self.conv(
            nn.functional.interpolate(
                x,size=self.out_shape,scale_factor=self.scale,mode='bilinear',align_corners=True))

In [21]:
def get_upSamp(feat_in,feat_out, out_shape=None, scale=2, act='relu'):
    
    upSamp = UpSample(feat_in,feat_out,out_shape=out_shape,scale=scale).cuda()
    
    layer = nn.Sequential(upSamp)
    
    if act == 'relu':
        act_f = nn.ReLU(inplace=True).cuda()
        bn = nn.BatchNorm2d(feat_out).cuda()
        layer.add_module('ReLU',act_f)
        layer.add_module('BN',bn)
    elif act == 'sig':
        act_f = nn.Sigmoid()
        layer.add_module('Sigmoid',act_f)
    return layer

def add_layer(m,feat_in,feat_out,name,out_shape=None,scale=2,act='relu'):
    upSamp = get_upSamp(feat_in,feat_out,out_shape=out_shape,scale=scale,act=act)
    m.add_module(name,upSamp)

In [22]:
add_layer(m,code_sz,256,'CodeOut',out_shape=(64,64),scale=None)

In [23]:
# m(b).size()

In [24]:
add_layer(m,256,128,'Upsample0')

In [25]:
# m(b).size()

In [26]:
add_layer(m,128,64,'Upsample1')

In [27]:
# m(b).size()
# m

In [28]:
add_layer(m,64,32,'Upsample2')

In [29]:
# m(b).size()

In [30]:
add_layer(m,32,1,'Upsample3',act='sig', scale=1)

In [31]:
m(b).size()
# b.size()

RuntimeError: CUDA out of memory. Tried to allocate 512.00 MiB (GPU 0; 7.79 GiB total capacity; 5.33 GiB already allocated; 109.00 MiB free; 5.91 GiB reserved in total by PyTorch)

In [36]:
learn = Learner(dls,m,loss_func=F.mse_loss)

In [37]:
# learn.final_record

In [38]:
# fastai.__version__
lr = 1e-2

In [39]:
learn.fit(10,lr)
# for i in dls.train_ds:
#     print(i.shape)
# count = 0
# for i in dls.train:
#     print(count+1)
#     print(i.shape)
#     count+=1

(#3) [0,0.0,'00:00']


RuntimeError: Expected 4-dimensional input for 4-dimensional weight [64, 1, 7, 7], but got 3-dimensional input of size [1, 512, 512] instead

In [75]:
fastai.__version__

'2.0.10'

In [76]:
m

Sequential(
  (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(1, 1))
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, 